In [33]:
from tqdm import tqdm
from datasketch import MinHash, MinHashLSH
from boltons.iterutils import windowed
from collections import defaultdict

from litecoder.models import City
from litecoder.us_cities import USCityKeyIter

In [18]:
def loc_to_mh(loc):
    
    mh = MinHash()
    
    for n in range(1, 5):
        for w in windowed(loc, n):
            mh.update(''.join(w).encode('utf8'))
            
    return mh

In [23]:
loc_to_mh('Washington, D.C.')

In [20]:
iter_keys = USCityKeyIter()

344249it [00:13, 25557.31it/s]


In [41]:
idx = MinHashLSH(0.8)

In [43]:
cities = City.query.filter(City.country_iso=='US')

idmap = dict()
i = 0
for city in tqdm(cities):
    
    try:

        # Generate keys, ensure no errors.
        keys = list(iter_keys(city))

        # Index complete key set.
        for key in keys:
            idx.insert(i, loc_to_mh(key))
            idmap[i] = city.wof_id
            i += 1

    except Exception as e:
        print(e)

7758it [03:38, 35.49it/s]

KeyboardInterrupt: 

In [58]:
for i in idx.query(loc_to_mh('I was born in tuscaloosa al')):
    print(City.query.get(idmap[i]))